In [ ]:
!pip install torch_geometric > /dev/null

ERROR: Operation cancelled by user


In [ ]:
import torch
import torch_geometric
import copy
from torch_geometric.datasets import AMiner
from torch_geometric.nn import MetaPath2Vec

In [ ]:
class Args:
  def __init__(self):
    self.root_dir = "/content"
    self.data_dir = "/content/data"
    self.epochs = 300
    self.runs = 5
    self.droput = 0.4
    self.lr = 0.001
    self.wd = 0.001
    self.num_layers = 2
    self.num_hidden = 256
    self.num_features = 0 # placeholder
    self.num_classes = 0 # placeholder

def add_data_features(args, data):
  args.num_features = data.x.shape[1]
  args.num_classes = data.y.shape[0]
  return args

In [ ]:
args = Args()
dataset = AMiner(root=args.root_dir)
data = dataset[0]
print(data)

HeteroData(
  author={
    y=[246678],
    y_index=[246678],
    num_nodes=1693531,
  },
  venue={
    y=[134],
    y_index=[134],
    num_nodes=3883,
  },
  paper={ num_nodes=3194405 },
  (paper, written_by, author)={ edge_index=[2, 9323605] },
  (author, writes, paper)={ edge_index=[2, 9323605] },
  (paper, published_in, venue)={ edge_index=[2, 3194405] },
  (venue, publishes, paper)={ edge_index=[2, 3194405] }
)


In [ ]:
"""
new_data = copy.deepcopy(data)
#print(new_data.edge_types)
print(new_data.edge_types[0])


print(data.edge_index_dict)
"""

'\nnew_data = copy.deepcopy(data)\n#print(new_data.edge_types)\nprint(new_data.edge_types[0])\n\n\nprint(data.edge_index_dict)\n'

In [ ]:
  # We can see by addition that all the edges are now in the same dict, so we add that dict to the new data
def make_homogeneous(data, new_data):
    all_edges = torch.hstack((data.edge_index_dict[data.edge_types[0]], data.edge_index_dict[data.edge_types[1]], data.edge_index_dict[data.edge_types[2]], data.edge_index_dict[data.edge_types[3]]))
    print(f"Shape of new edges: {all_edges.shape}")
    print("Shapes of old edges add to new shape: ")
    print(data.edge_index_dict[data.edge_types[0]].shape)
    print(data.edge_index_dict[data.edge_types[1]].shape)
    print(data.edge_index_dict[data.edge_types[2]].shape)
    print(data.edge_index_dict[data.edge_types[3]].shape)
    print()

    print("Deleting original edges...")
    for edge_type in new_data.edge_types:
        success = new_data.remove_edge_index(edge_type=edge_type, layout='coo')
        if success:
            print(f"Successfully removed edge index for {edge_type}")
        else:
            print(f"Failed to remove edge index for {edge_type}")

    del(new_data['paper', 'written_by', 'author'])
    del(new_data['author', 'writes', 'paper'])
    del(new_data['paper', 'published_in', 'venue'])
    del(new_data['venue', 'publishes', 'paper'])

    print(f"new_data has old edges removed: {new_data}")
    print("Adding new edges...")
    new_data.edge_index_dict = all_edges
    print(f"Resulting Data Structure: {new_data}")


In [ ]:
# to_homogeneous() method seems to double the connections but when I only pick one sided connections I get an error...
#make_homogeneous(data, new_data)

In [ ]:
homogeneous_data = data.to_homogeneous()

In [ ]:
homogeneous_data.edge_index_dict = {('node_type', 'edge_type', 'node_type') : homogeneous_data.edge_index}

In [ ]:
homogeneous_data

Data(
  edge_index=[2, 25036020],
  y=[3441217],
  y_index=[3441217],
  node_type=[4891819],
  edge_type=[25036020],
  edge_index_dict={ (node_type, edge_type, node_type)=[2, 25036020] }
)

In [ ]:
from torch_geometric.nn import MetaPath2Vec

# defining all 'metapaths' as tuples
metapath = [
    ('node_type', 'edge_type', 'node_type')
]

# select device and display it to user
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"DEVICE USED IN THIS SESSION: {device}")


model = MetaPath2Vec(homogeneous_data.edge_index_dict, embedding_dim=128,
                     metapath=metapath, walk_length=50, context_size=7,
                     walks_per_node=5, num_negative_samples=5,
                     sparse=True).to(device)

loader = model.loader(batch_size=128, shuffle=True, num_workers=2)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)


def train(epoch, log_steps=100, eval_steps=2000):
    model.train()

    total_loss = 0
    for i, (pos_rw, neg_rw) in enumerate(iter(loader)):
        #print(pos_rw.shape)
        #print(neg_rw.shape)
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        if (i + 1) % log_steps == 0:
            print((f'Epoch: {epoch}, Step: {i + 1:05d}/{len(loader)}, '
                   f'Loss: {total_loss / log_steps:.4f}'))
            total_loss = 0

        if (i + 1) % eval_steps == 0:
            acc = train_acc()
            print((f'Epoch: {epoch}, Step: {i + 1:05d}/{len(loader)}, '
                   f'Acc: {acc:.4f}'))


def train_acc(train_ratio=0.1):
    model.eval()

    z = model('node_type', batch=data['author'].y_index.to(device))
    y = data['author'].y

    perm = torch.randperm(z.size(0))
    train_perm = perm[:int(z.size(0) * train_ratio)]
    test_perm = perm[int(z.size(0) * train_ratio):]

    return model.test(z[train_perm], y[train_perm], z[test_perm], y[test_perm],
                      max_iter=150)

def test_acc(train_ratio = 0.1):
    model.eval()
    z = model('node_type', batch=data['author'].y_index.to(device))
    y = data['author'].y

    perm = torch.randperm(z.size(0))
    train_perm = perm[:int(z.size(0) * train_ratio)]
    test_perm = perm[int(z.size(0) * train_ratio):]

    return model.test(z[test_perm], y[test_perm], z[test_perm], y[test_perm],
                      max_iter=150)


for epoch in range(1, 5):
  train(epoch)
  print(f"Epoch: {epoch} | Train Accuracy: {train_acc():.4f} | Test Accuracy: {test_acc():.4f}")

DEVICE USED IN THIS SESSION: cuda
Epoch: 1, Step: 00100/38218, Loss: 9.6198
Epoch: 1, Step: 00200/38218, Loss: 9.2327
Epoch: 1, Step: 00300/38218, Loss: 8.7798
Epoch: 1, Step: 00400/38218, Loss: 8.3400
Epoch: 1, Step: 00500/38218, Loss: 7.9299
Epoch: 1, Step: 00600/38218, Loss: 7.5603
Epoch: 1, Step: 00700/38218, Loss: 7.2234
Epoch: 1, Step: 00800/38218, Loss: 6.9146
Epoch: 1, Step: 00900/38218, Loss: 6.6357
Epoch: 1, Step: 01000/38218, Loss: 6.3699
Epoch: 1, Step: 01100/38218, Loss: 6.1310
Epoch: 1, Step: 01200/38218, Loss: 5.9049
Epoch: 1, Step: 01300/38218, Loss: 5.6961
Epoch: 1, Step: 01400/38218, Loss: 5.4982
Epoch: 1, Step: 01500/38218, Loss: 5.3116
Epoch: 1, Step: 01600/38218, Loss: 5.1362
Epoch: 1, Step: 01700/38218, Loss: 4.9723
Epoch: 1, Step: 01800/38218, Loss: 4.8169
Epoch: 1, Step: 01900/38218, Loss: 4.6654
Epoch: 1, Step: 02000/38218, Loss: 4.5223
Epoch: 1, Step: 02000/38218, Acc: 0.2765
Epoch: 1, Step: 02100/38218, Loss: 4.3881
Epoch: 1, Step: 02200/38218, Loss: 4.2603
E

KeyboardInterrupt: 